d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# 3.4 File Formats

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Compare file formats and compression types
* Examine Delta

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Comparing File Formats<br>

Let's take a look at a colon delimited file sitting on S3.<br>
The size of the file is approximately **1.9GB**

In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-colon.txt

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-colon.txt,fire-calls-colon.txt,1829688314


Take a look at the first few lines of the file.

In [0]:
%fs head --maxBytes=1000 /mnt/davis/fire-calls/fire-calls-colon.txt

Create a temporary view of the file using `:` as the separator.<br>
Here the table creation is not infering the schema.Due to this the execution time is fast(**0.6 seconds**)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-colon.txt",
    header "true",
    sep ":"
  )

Take a look at the types of data in the table.<br>
**All the data are treated like strings. Strong Integer values as string is not efficient at all**

In [0]:
%sql
DESCRIBE fireCallsCSV

col_name,data_type,comment
Call Number,string,null
Unit ID,string,null
Incident Number,string,null
Call Type,string,null
Call Date,string,null
Watch Date,string,null
Received DtTm,string,null
Entry DtTm,string,null
Dispatch DtTm,string,null
Response DtTm,string,null


Are these data types correct? All of them are string types.

We need to tell Spark to infer the schema.<br>
**When we infer a schema the execution time reduces because checking the schema of a file is expensive operation**<br>
It took **1 minute 22 seconds** where as without infering the schema it was **0.6 seconds**

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-colon.txt",
    header "true",
    sep ":",
    inferSchema "true"
  )

Now take a look at how Spark inferred the data types.

In [0]:
%sql
DESCRIBE fireCallsCSV

col_name,data_type,comment
Call Number,int,null
Unit ID,string,null
Incident Number,int,null
Call Type,string,null
Call Date,string,null
Watch Date,string,null
Received DtTm,string,null
Entry DtTm,string,null
Dispatch DtTm,string,null
Response DtTm,string,null


Wow, that took a long time just to figure out the schema for this file! 

Now let's try the same thing with compressed files (Gzip and Bzip formats).

Notice that the bzip file is the most compact - we will see if it is the fastest to operate on.

In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-colon.txt

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-colon.txt,fire-calls-colon.txt,1829688314


In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-1p.txt.gz

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_SUCCESS,_SUCCESS,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_committed_1074000646993103229,_committed_1074000646993103229,116
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/_started_1074000646993103229,_started_1074000646993103229,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.gz/part-00000-tid-4222569572634380978-7f3cb922-b07c-4d71-b14f-f65d838c499a-15121-c000.csv.gz,part-00000-tid-4222569572634380978-7f3cb922-b07c-4d71-b14f-f65d838c499a-15121-c000.csv.gz,259566857


In [0]:
%fs ls /mnt/davis/fire-calls/fire-calls-1p.txt.bzip

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_SUCCESS,_SUCCESS,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_committed_7012599954241226118,_committed_7012599954241226118,117
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/_started_7012599954241226118,_started_7012599954241226118,0
dbfs:/mnt/davis/fire-calls/fire-calls-1p.txt.bzip/part-00000-tid-7012599954241226118-f407e362-c5a4-49ec-973b-3420b6ae94fd-15136-c000.csv.bz2,part-00000-tid-7012599954241226118-f407e362-c5a4-49ec-973b-3420b6ae94fd-15136-c000.csv.bz2,192991035


**The file size reduced from 1.9GB in its raw format into 192MB when we use bzip and 260MB with gz file compression techniques**

Let's start by reading in the gzipped file.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSVgzip
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.txt.gz",
    header "true",
    sep ":",
    inferSchema "true"
  )

Wow! That took way longer than inferring the schema on the uncompressed data. Even though it took up less storage space, we had to pay for that in computation.

You'll notice that the resulting view is comprised of only 1 partition, which makes this data very slow to query later on.

In [0]:
%python
sql("SELECT * FROM fireCallsCSVgzip").rdd.getNumPartitions()

Out[2]: 1

Let's compare the speed of reading in the gzip file to the bzip file!This takes long compared to the gzip one because this reads from multiple partitions

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSVbzip
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.txt.bzip",
    header "true",
    sep ":",
    inferSchema "true"
  )

In [0]:
%python
sql("SELECT * FROM fireCallsCSVbzip").rdd.getNumPartitions()

Out[3]: 8

Bzip is a "splittable" file format, so it is much better to use than gzip when working with row-based formats for querying later on.

Now let's go ahead and compare that to reading in from a columnar format: Parquet.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsParquet
USING Parquet 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.parquet"
  )

In [0]:
%sql
DESCRIBE fireCallsParquet

col_name,data_type,comment
Call_Number,int,null
Unit_ID,string,null
Incident_Number,int,null
Call_Type,string,null
Call_Date,string,null
Watch_Date,string,null
Received_DtTm,string,null
Entry_DtTm,string,null
Dispatch_DtTm,string,null
Response_DtTm,string,null


Look at how fast it is to get the schema from a Parquet file! That is because the Parquet file stores the data and the associated metadata.

Compare the performance between the three file types. We are going to use a Python helper function called [timeit](https://ipython.org/ipython-doc/3/interactive/magics.html#magic-timeit) to calculate how long the query takes to execute.

In [0]:
%python
parquetDF = sql("SELECT * FROM fireCallsParquet")
%timeit -n1 -r1 parquetDF.select("City").where("City == 'San Francisco'").count()

1.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [0]:
%python
csvDF = sql("SELECT * FROM fireCallsCSV")
%timeit -n1 -r1 csvDF.select("City").where("City == 'San Francisco'").count()

48.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [0]:
%python
gzipDF = sql("SELECT * FROM fireCallsCSVgzip")
%timeit -n1 -r1 gzipDF.select("City").where("City == 'San Francisco'").count()

24.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [0]:
%python
bzipDF = sql("SELECT * FROM fireCallsCSVbzip")
%timeit -n1 -r1 bzipDF.select("City").where("City == 'San Francisco'").count()

1min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Reading from the parquet storage is faster compared to the CSV file and bzip and gzip compressed files.**<br>
**Note: the bzip is the slowest of all** <br>

The execution time ranges from **1.6 seconds** to **1.6 minutes**

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Reading from Delta Files

"Apache Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language."

<div style="text-align:right">
![parquet logo](https://parquet.apache.org/assets/img/parquet_logo.png)<br>
<a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a></div>

-sandbox
Delta is built on Parquet, is also open source, and provides certain benefits beyond Parquet (more on this in later notebooks).

<div style="text-align:right">
![delta logo](https://live-delta-io.pantheonsite.io/wp-content/uploads/2019/04/delta-lake-logo-tm.png)<br>

### About Delta Files
* Free & Open Source.
* Increased query performance over row-based data stores.
* Provides efficient data compression.
* Designed for performance on large data sets.
* Supports schema evolution.
* Is a splittable "file format".
* A <a href="https://en.wikipedia.org/wiki/Column-oriented_DBMS" target="_blank">Column-Oriented</a> data store

**Row Format**

| ID | Name | Score |
|----|------|-------|
| 1 | john | 4.1 |
| 2 | mike | 3.5 |
| 3 | sally | 6.4 |

**Column Format**

| ID | 1 | 2 | 3 |
|----|------|-------|-------|
| Name | john | mike | sally |
| Score | 4.1 | 3.5 | 6.4 |

See also


* <a href="https://delta.io/" target="_blank">https&#58;//delta.io/</a>
* <a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a>
* <a href="https://en.wikipedia.org/wiki/Apache_Parquet" target="_blank">https&#58;//en.wikipedia.org/wiki/Apache_Parquet</a>

Write out to a delta table and repartition into 8 partitions.<br>
*/*+ REPARTITION(8) */* * do the repartitioning

In [0]:
%sql
DROP TABLE IF EXISTS fireCallsDelta;

CREATE TABLE fireCallsDelta
USING delta
AS
  SELECT /*+ REPARTITION(8) */ *
  FROM fireCallsParquet

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED fireCallsDelta

col_name,data_type,comment
Call_Number,int,
Unit_ID,string,
Incident_Number,int,
Call_Type,string,
Call_Date,string,
Watch_Date,string,
Received_DtTm,string,
Entry_DtTm,string,
Dispatch_DtTm,string,
Response_DtTm,string,


In [0]:
%python
dbutils.fs.ls("/user/hive/warehouse/databricks.db/firecallsdelta")

Now compare reading from a CSV file to reading from Delta.

In [0]:
%sql
SELECT count(`Incident Number`)
FROM firecallsCSV
WHERE Priority > 1

count(Incident Number)
4401920


In [0]:
%sql
SELECT count(`Incident_Number`)
FROM fireCallsDelta
WHERE Priority > 1

count(Incident_Number)
4401920


The Delta file has 8 partitions rather than 1. Look at the speed improvement!<br>
**Reading from the delat table is faster.**

##Reading CSV
- `spark.read.csv(..)`
- There are a large number of options when reading CSV files including headers, column separator, escaping, etc.
- We can allow Spark to infer the schema at the cost of first reading in the entire file
- Large CSV files should always have a schema pre-defined

## Reading Parquet
- `spark.read.parquet(..)`
- Parquet files are the preferred file format for big-data
- It is a columnar file format
- It is a splittable file format
- It offers a lot of performance benefits over other formats including predicate push down
- Unlike CSV, the schema is read in, not inferred
- Reading the schema from Parquet's metadata can be extremely efficient

-sandbox

## Comparison
| Type    | <span style="white-space:nowrap">Inference Type</span> | <span style="white-space:nowrap">Inference Speed</span> | Reason                                          | <span style="white-space:nowrap">Should Supply Schema?</span> |
|---------|--------------------------------------------------------|---------------------------------------------------------|----------------------------------------------------|:--------------:|
| <b>CSV</b>     | <span style="white-space:nowrap">Full-Data-Read</span> | <span style="white-space:nowrap">Slow</span>            | <span style="white-space:nowrap">File size</span>  | Yes            |
| <b>Parquet</b> | <span style="white-space:nowrap">Metadata-Read</span>  | <span style="white-space:nowrap">Fast/Medium</span>     | <span style="white-space:nowrap">Number of Partitions</span> | No (most cases)             |
| <b>Tables</b>  | <span style="white-space:nowrap">n/a</span>            | <span style="white-space:nowrap">n/a</span>            | <span style="white-space:nowrap">Predefined</span> | n/a            |
| <b>JSON</b>    | <span style="white-space:nowrap">Full-Read-Data</span> | <span style="white-space:nowrap">Slow</span>            | <span style="white-space:nowrap">File size</span>  | Yes            |
| <b>Text</b>    | <span style="white-space:nowrap">Dictated</span>       | <span style="white-space:nowrap">Zero</span>            | <span style="white-space:nowrap">Only 1 Column</span>   | Never          |
| <b>JDBC</b>    | <span style="white-space:nowrap">DB-Read</span>        | <span style="white-space:nowrap">Fast</span>            | <span style="white-space:nowrap">DB Schema</span>  | No             |

## Reading Tables
- `spark.read.table(..)`
- The Databricks platform allows us to register a huge variety of data sources as tables via the Databricks UI
- Any `DataFrame` (from CSV, Parquet, whatever) can be registered as a temporary view
- Tables/Views can be loaded via the `DataFrameReader` to produce a `DataFrame`
- Tables/Views can be used directly in SQL statements

## Reading JSON
- `spark.read.json(..)`
- JSON represents complex data types unlike CSV's flat format
- Has many of the same limitations as CSV (needing to read the entire file to infer the schema)
- Like CSV has a lot of options allowing control on date formats, escaping, single vs. multiline JSON, etc.

## Reading Text
- `spark.read.text(..)`
- Reads one line of text as a single column named `value`
- Is the basis for more complex file formats such as fixed-width text files

## Reading JDBC
- `spark.read.jdbc(..)`
- Requires one database connection per partition
- Has the potential to overwhelm the database
- Requires specification of a stride to properly balance partitions

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>